# Sequence alteration in protein complex
This notebook aims to explore what outcomes a single amino acid mutation in MMACHC can produce in the complex made of MMACHC, MMADHC, MTR and MTRR. These proteins are all implied in vitamin B12 methabolism, and their role is crucial for the correct and healthy state of the organism. Mutations in MMACHC sequence cause Methylmalonic acidemia with homocystinuria, a rare genetic disease that occurs mostly in neonatal age. It is a methabolic disorder that causes severe defects to the organism, such as myocardiopathy, acidosis, neurological deterioration and anemia. Identifying genes related to this disease's manifestation may reduce its probability of occurring in childs that can be affected.

In this notebook we are using AlphaFold-Multimer output on the wild-type complex and the mutant MMACHC complexes to retrieve information about their predicted structures. Then, we are using FoldX to retrieve binding energies and information about complex stability. 

## Importing libraries

In [1]:
import json
import os
from io import StringIO


import re
import pandas as pd
import shutil
import numpy as np
import subprocess
from Bio.PDB import MMCIFParser, PDBParser, PDBIO
from scipy.stats import pearsonr
import glob
import time


# Importing AlphaFold files

In [2]:
# folders
DATA_DIR = "/home/luca3/Downloads"
ORIGINAL_DIR = "fold_complex_original"
MUTATION_DIR = "fold_complex_"
mutation_list = ["mut_pos90_vtoq",
                 "mut_pos196_ftoh",
                 "mut_pos197_ntoi",
                 "mut_pos202_dtoc",
                 "mut_pos234_ltoc",
                 "mut_pos206_rtoq"
                ]

MUTATION_REGEX = r"mut_pos(\d+)"  # extract mutation position from folder name
LOCAL_WINDOW = 5  # +/- residues

# json loader
def load_json(path):
    with open(path) as f:
        return json.load(f)

# metrics extractor
def extract_global_metrics(full_data_path, summary_path):
    # Get mean pLDDT from atom_plddts
    full_data = load_json(full_data_path)
    residues = full_data.get("atom_plddts", [])
    mean_plddt = sum(residues) / len(residues) if residues else None

    # PTM score from summary
    summary_data = load_json(summary_path)
    ptm_score = summary_data.get("ptm")  # or None if not there

    return mean_plddt, ptm_score

# local plddt in loc of mutation
def extract_local_plddt(full_data_path, pos, window):
    data = load_json(full_data_path)
    residues = data.get("atom_plddts", [])
    start = max(0, pos - 1 - window)
    end = min(len(residues), pos + window)
    local_scores = residues[start:end]
    return sum(local_scores) / len(local_scores) if local_scores else None

# Cα coordinates extractor from CIF file
def extract_ca_coords_cif(cif_file):
    parser = MMCIFParser()
    structure = parser.get_structure("prot", cif_file)
    coords = {}
    for model in structure:
        for chain in model:
            chain_id = chain.id
            ca_coords = []
            for res in chain:
                if "CA" in res:
                    ca_coords.append(res["CA"].get_coord())
            coords[chain_id] = np.array(ca_coords)
        break  # only first model
    return coords

# Effective Strain extractor
def effective_strain_vector(ca_coords_ref, ca_coords_target, cutoff=13.0):
    n_residues = ca_coords_ref.shape[0]
    es_vector = np.zeros(n_residues)
    
    for i in range(n_residues):
        ref_coord = ca_coords_ref[i]
        distances = np.linalg.norm(ca_coords_ref - ref_coord, axis=1)
        neighbors = np.where(distances <= cutoff)[0]
        diff = ca_coords_target[neighbors] - ca_coords_ref[neighbors]
        es_vector[i] = np.sqrt(np.mean(np.sum(diff**2, axis=1)))
    
    return es_vector


# pearson correlation between Effective strains
def es_correlation(ca_coords_ref, ca_coords_target, cutoff=13.0):
    es_ref = effective_strain_vector(ca_coords_ref, ca_coords_ref, cutoff)
    es_target = effective_strain_vector(ca_coords_ref, ca_coords_target, cutoff)
    
    # Only compute correlation if es_ref is not constant
    if np.std(es_ref) == 0 or np.std(es_target) == 0:
        r = np.nan
    else:
        r, _ = pearsonr(es_ref, es_target)
    return r, es_ref, es_target

# effective strain computer in zone of mutation
def effective_strain_ca(ca_coords_wt, ca_coords_mut, mut_idx, cutoff=13.0):
    ref_coord = ca_coords_wt[mut_idx]
    distances = np.linalg.norm(ca_coords_wt - ref_coord, axis=1)
    neighbor_idx = np.where(distances <= cutoff)[0]

    # Compute RMSD in the neighborhood
    diff = ca_coords_mut[neighbor_idx] - ca_coords_wt[neighbor_idx]
    es = np.sqrt(np.mean(np.sum(diff**2, axis=1)))
    return es

    

In [3]:
# load files of wild-type
orig_summary_file = os.path.join(DATA_DIR, ORIGINAL_DIR, f"{ORIGINAL_DIR}_summary_confidences_0.json")
orig_full_file = os.path.join(DATA_DIR, ORIGINAL_DIR, f"{ORIGINAL_DIR}_full_data_0.json")
orig_mean_plddt, orig_ptm = extract_global_metrics(orig_full_file, orig_summary_file)

# load mutants

results = []

for mut in mutation_list:
    mut_dir = MUTATION_DIR + mut
    folder_path = os.path.join(DATA_DIR, mut_dir)
    print(f'Obtaining file at: {folder_path}')
    name_match = re.search(MUTATION_REGEX, mut)
    if not name_match:
        print(f"⚠️ Skipping {mut} — position not found in name")
        continue
    mut_pos = int(name_match.group(1))
    mut_idx = mut_pos - 1  # 0-based index

    # storage for per-model metrics
    per_model_metrics = {
        "mean_plddt": [],
        "tm_score": [],
        "local_plddt": [],
        "local_es": [],
        "avg_es": []
    }

    # Loop over all 5 models
    for model_idx in range(5):
        summary_file = os.path.join(folder_path, f"{mut_dir}_summary_confidences_{model_idx}.json")
        full_data_file = os.path.join(folder_path, f"{mut_dir}_full_data_{model_idx}.json")

        if not os.path.exists(summary_file) or not os.path.exists(full_data_file):
            print(f"⚠️ Missing files for {mut}, model {model_idx}")
            continue

        mean_plddt, tm_score = extract_global_metrics(full_data_file, summary_file)
        local_plddt_mut = extract_local_plddt(full_data_file, mut_pos, LOCAL_WINDOW)

        wt_cif_file = os.path.join(DATA_DIR, ORIGINAL_DIR, f"{ORIGINAL_DIR}_model_{model_idx}.cif")
        mut_cif_file = os.path.join(folder_path, f"{mut_dir}_model_{model_idx}.cif")

        wt_coords = extract_ca_coords_cif(wt_cif_file)["A"]  # assuming chain A
        mut_coords = extract_ca_coords_cif(mut_cif_file)["A"]

        r_es, es_wt_vec, es_mut_vec = es_correlation(wt_coords, mut_coords, cutoff=13.0)
        local_es_mut = effective_strain_ca(wt_coords, mut_coords, mut_idx, cutoff=13.0)

        # collect
        per_model_metrics["mean_plddt"].append(mean_plddt)
        per_model_metrics["tm_score"].append(tm_score)
        per_model_metrics["local_plddt"].append(local_plddt_mut)
        per_model_metrics["local_es"].append(local_es_mut)
        per_model_metrics["avg_es"].append(np.mean(es_mut_vec))

    # Average over models (skip None)
    avg_mean_plddt = np.nanmean([m for m in per_model_metrics["mean_plddt"] if m is not None])
    avg_tm_score = np.nanmean([m for m in per_model_metrics["tm_score"] if m is not None])
    avg_local_plddt = np.nanmean([m for m in per_model_metrics["local_plddt"] if m is not None])
    avg_local_es = np.nanmean([m for m in per_model_metrics["local_es"] if m is not None])
    avg_avg_es = np.nanmean([m for m in per_model_metrics["avg_es"] if m is not None])

    # compute std
    sd_mean_plddt = np.nanstd([m for m in per_model_metrics["mean_plddt"] if m is not None])
    sd_tm_score = np.nanstd([m for m in per_model_metrics["tm_score"] if m is not None])
    sd_local_plddt = np.nanstd([m for m in per_model_metrics["local_plddt"] if m is not None])
    sd_local_es = np.nanstd([m for m in per_model_metrics["local_es"] if m is not None])
    sd_avg_es = np.nanstd([m for m in per_model_metrics["avg_es"] if m is not None])

    # Original (WT) values — averaged across the same 5 models
    orig_per_model_plddt = []
    orig_per_model_tm = []
    orig_per_model_local_plddt = []
    for model_idx in range(5):
        orig_full_data_file = os.path.join(DATA_DIR, ORIGINAL_DIR, f"{ORIGINAL_DIR}_full_data_{model_idx}.json")
        if os.path.exists(orig_full_data_file):
            mean_plddt_orig, tm_score_orig = extract_global_metrics(
                orig_full_data_file,
                os.path.join(DATA_DIR, ORIGINAL_DIR, f"{ORIGINAL_DIR}_summary_confidences_{model_idx}.json")
            )
            orig_per_model_plddt.append(mean_plddt_orig)
            orig_per_model_tm.append(tm_score_orig)
            orig_per_model_local_plddt.append(
                extract_local_plddt(orig_full_data_file, mut_pos, LOCAL_WINDOW)
            )

    avg_mean_plddt_orig = np.nanmean(orig_per_model_plddt)
    avg_tm_score_orig = np.nanmean(orig_per_model_tm)
    avg_local_plddt_orig = np.nanmean(orig_per_model_local_plddt)

    # stds
    std_mean_plddt_orig = np.nanstd(orig_per_model_plddt)
    std_tm_score_orig = np.nanstd(orig_per_model_tm)
    std_local_plddt_orig = np.nanstd(orig_per_model_local_plddt)

    results.append({
        "mutation": mut,
        "mut_pos": mut_pos,
        "mean_plddt_mut": avg_mean_plddt,
        "std_plddt_mut": sd_mean_plddt / np.sqrt(5),
        "mean_plddt_orig": avg_mean_plddt_orig,
        "delta_mean_plddt": avg_mean_plddt - avg_mean_plddt_orig if not np.isnan(avg_mean_plddt) and not np.isnan(avg_mean_plddt_orig) else None,
        "tm_score_mut": avg_tm_score,
        "std_tm_score_mut": sd_tm_score/ np.sqrt(5),
        "tm_score_orig": avg_tm_score_orig,
        "delta_tm_score": avg_tm_score - avg_tm_score_orig if not np.isnan(avg_tm_score) and not np.isnan(avg_tm_score_orig) else None,
        "local_plddt_mut": avg_local_plddt,
        "std_local_plddt_mut": sd_local_plddt/ np.sqrt(5),
        "local_plddt_orig": avg_local_plddt_orig,
        "delta_local_plddt": avg_local_plddt - avg_local_plddt_orig if not np.isnan(avg_local_plddt) and not np.isnan(avg_local_plddt_orig) else None,
        "local_es_mut": avg_local_es,
        "std_local_es_mut": sd_local_es/ np.sqrt(5),
        "avg_es_mut": avg_avg_es,
        "std_es_mut": sd_avg_es/ np.sqrt(5),
        "local_es_orig": 0.0,
        "delta_es": avg_local_es - 0.0
    })


# Save results
df = pd.DataFrame(results)
df.to_csv("mutations_stability_comparison_avg_over_models.csv", index=False)
df


Obtaining file at: /home/luca3/Downloads/fold_complex_mut_pos90_vtoq
Obtaining file at: /home/luca3/Downloads/fold_complex_mut_pos196_ftoh
Obtaining file at: /home/luca3/Downloads/fold_complex_mut_pos197_ntoi
Obtaining file at: /home/luca3/Downloads/fold_complex_mut_pos202_dtoc
Obtaining file at: /home/luca3/Downloads/fold_complex_mut_pos234_ltoc
Obtaining file at: /home/luca3/Downloads/fold_complex_mut_pos206_rtoq


,mutation,mut_pos,mean_plddt_mut,std_plddt_mut,mean_plddt_orig,delta_mean_plddt,tm_score_mut,std_tm_score_mut,tm_score_orig,delta_tm_score,local_plddt_mut,std_local_plddt_mut,local_plddt_orig,delta_local_plddt,local_es_mut,std_local_es_mut,avg_es_mut,std_es_mut,local_es_orig,delta_es
0,mut_pos90_vtoq,90,75.336901,0.163803,75.472173,-0.135272,0.498,0.001789,0.498,0.000,78.112182,0.242842,79.866727,-1.754545,51.094109,5.244840,57.082565,3.766440,0.0,51.094109
1,mut_pos196_ftoh,196,74.998069,0.185753,75.472173,-0.474104,0.482,0.001789,0.498,-0.016,85.323636,0.246239,85.646545,-0.322909,65.827538,9.488998,64.461572,6.899402,0.0,65.827538
2,mut_pos197_ntoi,197,75.338517,0.045887,75.472173,-0.133656,0.498,0.001789,0.498,0.000,85.560545,0.154084,86.006727,-0.446182,66.915176,9.289279,64.449314,7.910210,0.0,66.915176
3,mut_pos202_dtoc,202,75.517971,0.096021,75.472173,0.045798,0.492,0.001789,0.498,-0.006,89.290364,0.261721,89.083273,0.207091,51.779499,8.738703,56.879727,6.552232,0.0,51.779499
4,mut_pos234_ltoc,234,75.604506,0.196223,75.472173,0.132333,0.500,0.000000,0.498,0.002,86.327273,0.411635,86.797091,-0.469818,49.026478,9.617260,57.466028,9.338823,0.0,49.026478
5,mut_pos206_rtoq,206,75.036774,0.105276,75.472173,-0.435399,0.486,0.002191,0.498,-0.012,86.785818,0.166037,87.398364,-0.612545,65.963913,11.106281,58.657677,7.935801,0.0,65.963913


# FoldX code

In [4]:
# silence warning
import warnings 

warnings.filterwarnings(
    "ignore",
    message="The 'delim_whitespace' keyword in pd.read_csv is deprecated"
)


In [5]:
import os
import glob
import shutil
import subprocess
import pandas as pd
from Bio.PDB import MMCIFParser, PDBIO

# Paths
DATA_DIR = "/home/luca3/Downloads"
OUTPUT_DIR = os.path.join(DATA_DIR, "foldx_results")
os.makedirs(OUTPUT_DIR, exist_ok=True)

foldx_bin = '/home/luca3/Desktop/foldx5_1Linux64/foldx'

# ---- Core helpers ----
def run_foldx(command, pdb_file, output_dir=OUTPUT_DIR, extra_args=None):
    """
    Run FoldX with the given command.
    pdb_file: full path to pdb inside OUTPUT_DIR.
    """
    pdb_name = os.path.basename(pdb_file)
    foldx_cmd = [
        foldx_bin,
        f"--command={command}",
        f"--pdb={pdb_name}",
        f"--output-dir={output_dir}"
    ]
    if extra_args:
        foldx_cmd.extend(extra_args)

    print(f"Attempting {command} on {pdb_name}")
    start_time = time.time()
    # WITH FOLDX output
    #subprocess.run(foldx_cmd, cwd=output_dir)

    # WITHOUT FOLDX output
    
    subprocess.run(foldx_cmd, 
                   cwd= output_dir,
                   stdout=subprocess.DEVNULL, # suppress standard output 
                   stderr=subprocess.DEVNULL # suppress error output 
                  )
    end_time = time.time()
    print(f"Time needed for {command}: {round(end_time-start_time,2)}s")

def cif_to_pdb(cif_file, output_dir=OUTPUT_DIR):
    """Convert CIF to PDB in OUTPUT_DIR."""
    parser = MMCIFParser()
    base_name = os.path.splitext(os.path.basename(cif_file))[0]
    structure = parser.get_structure(base_name, cif_file)

    pdb_file = os.path.join(output_dir, base_name + ".pdb")
    io = PDBIO()
    io.set_structure(structure)
    io.save(pdb_file)
    return pdb_file


def repair_pdb(pdb_file):
    pdb_name = os.path.basename(pdb_file)
    target_path = os.path.join(OUTPUT_DIR, pdb_name)
    repaired_name = pdb_name.replace(".pdb", "_Repair.fxout")
    repaired_path = os.path.join(OUTPUT_DIR, repaired_name)
    print(f"Repaired file full path should be {repaired_path}")

    if os.path.exists(repaired_path):
        print(f"Skipping repair for {pdb_name}, repaired PDB already exists.")
        return repaired_path

    if os.path.dirname(os.path.abspath(pdb_file)) != os.path.abspath(OUTPUT_DIR):
        shutil.copyfile(pdb_file, target_path)
        pdb_file = target_path

    print(f"Repairing {pdb_name}...")
    run_foldx("RepairPDB", pdb_file)

    if os.path.exists(repaired_path):
        return repaired_path
    else:
        return pdb_file


# ---- FoldX commands ----
def build_model(wt_pdb_file, mutant_list, n_models=5):
    """Run BuildModel for a given WT pdb and list of mutations."""
    wt_pdb_file = os.path.join(OUTPUT_DIR, os.path.basename(wt_pdb_file))
    mut_file_path = os.path.join(OUTPUT_DIR, "individual_list.txt")
    with open(mut_file_path, "w") as f:
        for mut in mutant_list:
            f.write(mut + ";\n")

    run_foldx("BuildModel", wt_pdb_file,
              extra_args=[f"--mutant-file={mut_file_path}", f"--numberOfRuns={n_models}"])


# ---- Parsers ----
def parse_stability_output(output_dir=OUTPUT_DIR):
    fxout_files = glob.glob(os.path.join(output_dir, "*_ST.fxout"))
    col_names = [
        "PDB", "Total_Energy", "Backbone_Hbond", "Sidechain_Hbond", 
        "Van_der_Waals", "Electrostatics", "Solvation_Polar", "Solvation_Hydrophobic",
        "VdWclashes", "entropy_sidechain", "entropy_mainchain", "Sloop", 
        "Shelix", "Sturn", "Ssolv", "intraclashes", "torsional_clash", "backbone_clash",
        "energy_extra1", "energy_extra2", "energy_extra3", "Unfolding_energy", 
        "Other1", "Other2"
    ]
    dfs = []
    for file in fxout_files:
        df = pd.read_csv(file, sep=r"\s+", header=None, engine="python")
        df.columns = col_names[:df.shape[1]]
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True) if dfs else None


def parse_analysecomplex_output(output_dir=OUTPUT_DIR):
    fxout_files = glob.glob(os.path.join(output_dir, "*_AC.fxout"))
    dfs = []

    # Column names for AnalyseComplex
    col_names = [
        "PDB", "Total_Energy", "Backbone_Hbond", "Sidechain_Hbond",
        "Van_der_Waals", "Electrostatics", "Solvation_Polar", "Solvation_Hydrophobic",
        "VdWclashes", "entropy_sidechain", "entropy_mainchain",
        "Sloop", "Shelix", "Sturn", "Ssolv",
        "intraclashes", "torsional_clash", "backbone_clash",
        "energy_extra1", "energy_extra2", "energy_extra3",
        "Binding_Energy",
        "Other1", "Other2"
    ]

    for file in fxout_files:
        # Skip FoldX header lines (start with # or Pdb)
        with open(file, "r") as f:
            lines = [l for l in f.readlines() if not l.startswith("#") and not l.startswith("Pdb")]

        # Parse with pandas, but allow inconsistent columns
        df = pd.read_csv(
            StringIO("".join(lines)),
            sep=r"\s+",
            engine="python",
            header=None,
            on_bad_lines="skip"
        )

        df.columns = col_names[:df.shape[1]]  # truncate/adjust column names
        dfs.append(df)

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        print("⚠️ No AnalyseComplex output files found")
        return None



# ---- Main pipeline ----
def process_cif_list(cif_list):
    all_results = []
    print(f"Processing {len(cif_list)} CIF files")
    for cif_file in cif_list:
        if not os.path.exists(cif_file):
            print(f"⚠ Missing CIF file: {cif_file}")
            continue

        pdb_file = cif_to_pdb(cif_file)
        repaired_pdb = repair_pdb(pdb_file)
        run_foldx("AnalyseComplex", repaired_pdb)
        df = parse_analysecomplex_output()
        if df is not None:
            all_results.append(df)

    return pd.concat(all_results, ignore_index=True) if all_results else None

def _normalize_cols(cols):
    return [re.sub(r'[^A-Za-z0-9_]+', '_', c).strip('_') for c in cols]

def parse_analysecomplex_output2(output_dir=OUTPUT_DIR):
    fxout_files = glob.glob(os.path.join(output_dir, "*_AC.fxout"))
    dfs = []
    for path in fxout_files:
        with open(path, "r") as f:
            lines = [ln.rstrip("\n") for ln in f]

        # indices of header lines that start blocks
        header_idxs = [i for i, ln in enumerate(lines) if ln.strip().startswith("Pdb")]

        for h in header_idxs:
            # end of this block = next header or EOF
            end = next((i for i in range(h+1, len(lines)) if lines[i].strip().startswith("Pdb")), len(lines))
            block = lines[h:end]

            # drop empty/dash/intro lines inside the block
            cleaned = []
            for ln in block:
                s = ln.strip()
                if not s:
                    continue
                if s.startswith("FoldX"):
                    continue
                if s.startswith("Output type"):
                    continue
                if set(s) == {"-"}:
                    continue
                cleaned.append(s)

            if len(cleaned) < 2:
                continue  # no data

            header = re.split(r"\s+", cleaned[0].strip())
            data_lines = cleaned[1:]
            csv_text = " ".join(header) + "\n" + "\n".join(data_lines)

            try:
                df = pd.read_csv(StringIO(csv_text), sep=r"\s+", engine="python")
            except Exception as e:
                print(f"⚠️ Skipping malformed block in {os.path.basename(path)}: {e}")
                continue

            df.columns = _normalize_cols(df.columns)
            dfs.append(df)

    if not dfs:
        print("⚠️ No valid AnalyseComplex tables found.")
        return None

    # Optional: add a canonical Binding_Energy column if present under another name
    alt_names =_
    df = pd.concat(dfs, ignore_index=True)
    return df

In [6]:
# MAIN PIPELINE

foldx_bin = '/home/luca3/Desktop/foldx5_1Linux64/foldx'
LOCAL_WINDOW = 5  # +/- residues

ORIGINAL_DIR = "fold_complex_original"
MUTATION_DIR = "fold_complex_"

# CIF base path (without model index)
ORIGINAL_CIF_PATTERN = os.path.join(DATA_DIR, "fold_complex_original", "fold_complex_original_model_{}.cif")
MODEL_RANGE = range(5)  # 0 to 4

CIFS_patterns = [] # list of cifs to take as inputs
# append original models paths
for model_idx in MODEL_RANGE:
    CIFS_patterns.append(ORIGINAL_CIF_PATTERN.format(model_idx))

# mutation paths 
for mut in mutation_list:
    folder_name = "fold_complex_" + mut
    model_name = "fold_complex_" + mut + "_model_{}.cif"
    MUT_CIF_PATTERN = os.path.join(DATA_DIR, folder_name, model_name)
    for model_idx in MODEL_RANGE:
        CIFS_patterns.append(MUT_CIF_PATTERN.format(model_idx))

# results saver
all_results = []
print(f"Inputted {len(CIFS_patterns)} CIF files")

for cif_file in CIFS_patterns:
    print(f"Obtaining file from path {cif_file}")
    if not os.path.exists(cif_file):
        print(f"⚠️ Missing CIF file for model {model_idx}: {cif_file}")
        continue

    # 1. Convert CIF → PDB
    pdb_file = cif_to_pdb(cif_file, OUTPUT_DIR)

    # 2. Repair PDB
    repaired_pdb = repair_pdb(pdb_file)

    # 3. Analyze with FoldX
    run_foldx("AnalyseComplex", os.path.basename(repaired_pdb))

    # 4. Parse results
    df = parse_analysecomplex_output2()
    all_results.append(df)
    print('\n')

Inputted 35 CIF files
Obtaining file from path /home/luca3/Downloads/fold_complex_original/fold_complex_original_model_0.cif
Repaired file full path should be /home/luca3/Downloads/foldx_results/fold_complex_original_model_0_Repair.fxout
Skipping repair for fold_complex_original_model_0.pdb, repaired PDB already exists.
Attempting AnalyseComplex on fold_complex_original_model_0_Repair.fxout
Time needed for AnalyseComplex: 20.29s


Obtaining file from path /home/luca3/Downloads/fold_complex_original/fold_complex_original_model_1.cif
Repaired file full path should be /home/luca3/Downloads/foldx_results/fold_complex_original_model_1_Repair.fxout
Skipping repair for fold_complex_original_model_1.pdb, repaired PDB already exists.
Attempting AnalyseComplex on fold_complex_original_model_1_Repair.fxout
Time needed for AnalyseComplex: 20.0s


Obtaining file from path /home/luca3/Downloads/fold_complex_original/fold_complex_original_model_2.cif
Repaired file full path should be /home/luca3/Down

In [7]:
# last element of all results contains everything
res = all_results[-1]
res

,Pdb,Group1,Group2,IntraclashesGroup1,IntraclashesGroup2,Interaction,Energy,StabilityGroup1,StabilityGroup2,Backbone,...,Residues_3,VdW,Clashing_1,Interface_3,Residues_4,BB,Clashing_2,Group,total,energy_1
0,./fold_complex_mut_pos90_vtoq_model_4.pdb,A,B,48.5047,133.925,63.969400,79.30320,262.608,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,./fold_complex_mut_pos90_vtoq_model_4.pdb,A,C,48.5047,339.355,51.281800,79.30320,287.654,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,./fold_complex_mut_pos90_vtoq_model_4.pdb,A,D,48.5047,162.329,48.431300,79.30320,263.516,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,./fold_complex_mut_pos90_vtoq_model_4.pdb,B,C,133.9250,339.355,945.390000,262.60800,287.654,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,./fold_complex_mut_pos90_vtoq_model_4.pdb,B,D,133.9250,162.329,17.210500,262.60800,263.516,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,./fold_complex_mut_pos202_dtoc_model_0_Repair.pdb,A,C,16.6300,89.970,120.872000,-7.84979,NaN,NaN,-24.439000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1036,./fold_complex_mut_pos202_dtoc_model_0_Repair.pdb,A,D,16.6300,29.240,-0.149242,-1.11342,NaN,NaN,-0.557611,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1037,./fold_complex_mut_pos202_dtoc_model_0_Repair.pdb,B,C,24.9500,89.970,-41.236600,-10.93820,NaN,NaN,-37.685700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1038,./fold_complex_mut_pos202_dtoc_model_0_Repair.pdb,B,D,24.9500,29.240,1.307080,-1.28273,NaN,NaN,-2.459050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# FoldX splits colnames badly, we need to fix it and collapse the columns
def fix_columns(df):
    cols = list(df.columns)
    merged = []
    new_data = []

    i = 0
    while i < len(cols):
        # Triple merge
        if i + 2 < len(cols) and cols[i:i+3] == ["Van", "der", "Waals"]:
            merged.append("Van_der_Waals")
            new_data.append(df.iloc[:, i])  # keep the first col’s data
            i += 3
        # Double merges
        elif i + 1 < len(cols) and cols[i:i+2] == ["Solvation", "Polar"]:
            merged.append("Solvation_Polar")
            new_data.append(df.iloc[:, i])
            i += 2
        elif i + 1 < len(cols) and cols[i:i+2] == ["Solvation_1", "Hydrophobic"]:
            merged.append("Solvation_Hydrophobic")
            new_data.append(df.iloc[:, i])
            i += 2
        elif i + 1 < len(cols) and cols[i:i+2] == ["torsional", "clash"]:
            merged.append("Torsional_Clash")
            new_data.append(df.iloc[:, i])
            i += 2
        elif i + 1 < len(cols) and cols[i:i+2] == ["backbone", "clash_1"]:
            merged.append("Backbone_Clash")
            new_data.append(df.iloc[:, i])
            i += 2
        elif i + 1 < len(cols) and cols[i:i+2] == ["entropy", "sidechain"]:
            merged.append("Entropy_Sidechain")
            new_data.append(df.iloc[:, i])
            i += 2
        elif i + 1 < len(cols) and cols[i:i+2] == ["entropy_1", "mainchain"]:
            merged.append("Entropy_Mainchain")
            new_data.append(df.iloc[:, i])
            i += 2
        elif i + 1 < len(cols) and cols[i:i+2] == ["VdW", "Clashing_1"]:
            merged.append("VdW_Clashing")
            new_data.append(df.iloc[:, i])
            i += 2
        elif i + 3 < len(cols) and cols[i:i+4] == ["Complex", "Number", "of", "Residues"]:
            merged.append("Complex_NumResidues")
            new_data.append(df.iloc[:, i])
            i += 4
        elif i + 1 < len(cols) and cols[i:i+2] == ["Interface", "Residues_1"]:
            merged.append("Interface_Residues")
            new_data.append(df.iloc[:, i])
            i += 2
        # Default: keep column as is
        else:
            merged.append(cols[i])
            new_data.append(df.iloc[:, i])
            i += 1

    # Build new DataFrame
    new_df = pd.concat(new_data, axis=1)
    new_df.columns = merged
    return new_df



res = fix_columns(res)
print(np.shape(res))
res.columns

(1040, 56)


Index(['Pdb', 'Group1', 'Group2', 'IntraclashesGroup1', 'IntraclashesGroup2',
       'Interaction', 'Energy', 'StabilityGroup1', 'StabilityGroup2',
       'Backbone', 'Hbond', 'Sidechain', 'Hbond_1', 'Van_der_Waals',
       'Electrostatics', 'Solvation_Polar', 'Solvation_Hydrophobic', 'Van_1',
       'der_1', 'Waals_1', 'clashes', 'Entropy_Sidechain', 'Entropy_Mainchain',
       'sloop_entropy', 'mloop_entropy', 'cis_bond', 'Torsional_Clash',
       'Backbone_Clash', 'helix', 'dipole', 'water', 'bridge', 'disulfide',
       'electrostatic', 'kon', 'partial', 'covalent', 'bonds', 'energy',
       'Ionisation', 'Entropy', 'Complex_NumResidues', 'Interface_Residues',
       'Interface_1', 'Residues_2', 'Clashing', 'Interface_2', 'Residues_3',
       'VdW_Clashing', 'Interface_3', 'Residues_4', 'BB', 'Clashing_2',
       'Group', 'total', 'energy_1'],
      dtype='object')

In [9]:
sel_cols = ['Pdb','Energy','Interaction','StabilityGroup1','StabilityGroup2','Interface_Residues','Clashing']
sel_res = res[sel_cols]

# groupby Pdb (average over all combinations of polypeptides)
sel_res = sel_res.groupby('Pdb').mean().reset_index()

sel_res

,Pdb,Energy,Interaction,StabilityGroup1,StabilityGroup2,Interface_Residues,Clashing
0,./fold_complex_mut_pos196_ftoh_model_0.pdb,67.203628,23.360328,227.496667,NaN,NaN,NaN
1,./fold_complex_mut_pos196_ftoh_model_0_Repair.pdb,-13.816666,-9.760897,-57.317767,NaN,NaN,NaN
2,./fold_complex_mut_pos196_ftoh_model_1.pdb,87.689023,90.001017,242.234167,NaN,NaN,NaN
3,./fold_complex_mut_pos196_ftoh_model_1_Repair.pdb,-3.426338,11.183540,-47.042050,NaN,NaN,NaN
4,./fold_complex_mut_pos196_ftoh_model_2.pdb,108.917289,250.876038,335.880833,NaN,NaN,NaN
...,...,...,...,...,...,...,...
60,./fold_complex_original_model_2_Repair.pdb,-4.122753,3.623492,-56.568200,NaN,NaN,NaN
61,./fold_complex_original_model_3.pdb,68.156048,67.919911,209.776333,NaN,NaN,NaN
62,./fold_complex_original_model_3_Repair.pdb,-3.794440,10.632232,-47.039100,NaN,NaN,NaN
63,./fold_complex_original_model_4.pdb,76.886189,30.504655,218.146167,NaN,NaN,NaN


In [10]:
# average over all possible models
positions = []
for i in range(np.shape(sel_res)[0]):
    positions.append((sel_res.iloc[i]['Pdb'][22:25]))

sel_res['Pos'] = positions
# select only numeric col
num_cols = sel_res.select_dtypes(include='number').columns
avg_df = sel_res.groupby('Pos')[num_cols].mean().reset_index()
std_df = sel_res.groupby('Pos')[num_cols].std().reset_index()
avg_df

,Pos,Energy,Interaction,StabilityGroup1,StabilityGroup2,Interface_Residues,Clashing
0,196,40.452035,67.597869,129.775040,NaN,NaN,NaN
1,197,33.853629,12.380530,72.459447,NaN,NaN,NaN
2,202,30.143481,17.803412,79.250286,NaN,NaN,NaN
3,206,-10.961263,-8.148512,-47.261611,NaN,NaN,NaN
4,234,35.769863,36.074602,84.467657,NaN,NaN,NaN
5,90_,32.618189,37.817639,99.052060,NaN,NaN,NaN
6,l_m,33.955446,15.630489,82.736097,NaN,NaN,NaN


In [11]:
std_df

,Pos,Energy,Interaction,StabilityGroup1,StabilityGroup2,Interface_Residues,Clashing
0,196,47.654140,79.255277,178.327451,NaN,NaN,NaN
1,197,41.198659,23.798264,138.247532,NaN,NaN,NaN
2,202,42.169155,28.743716,141.574066,NaN,NaN,NaN
3,206,1.799624,6.078785,21.844061,NaN,NaN,NaN
4,234,42.668342,55.553483,145.071729,NaN,NaN,NaN
5,90_,42.433861,59.690454,152.726696,NaN,NaN,NaN
6,l_m,40.294162,25.898951,142.688245,NaN,NaN,NaN


# FoldX Analysis skipping build model

In [12]:
all_results = []
for cif_file in CIFS_patterns:
    print(f"Obtaining file from path {cif_file}")
    if not os.path.exists(cif_file):
        print(f"⚠️ Missing CIF file for model {model_idx}: {cif_file}")
        continue

    # 1. Convert CIF → PDB
    pdb_file = cif_to_pdb(cif_file, OUTPUT_DIR)

    # 2. Repair PDB
    #repaired_pdb = repair_pdb(pdb_file)

    # 3. Analyze with FoldX
    run_foldx("AnalyseComplex", os.path.basename(pdb_file))

    # 4. Parse results
    df = parse_analysecomplex_output2()
    all_results.append(df)
    print('\n')

Obtaining file from path /home/luca3/Downloads/fold_complex_original/fold_complex_original_model_0.cif
Attempting AnalyseComplex on fold_complex_original_model_0.pdb
Time needed for AnalyseComplex: 19.51s


Obtaining file from path /home/luca3/Downloads/fold_complex_original/fold_complex_original_model_1.cif
Attempting AnalyseComplex on fold_complex_original_model_1.pdb
Time needed for AnalyseComplex: 19.45s


Obtaining file from path /home/luca3/Downloads/fold_complex_original/fold_complex_original_model_2.cif
Attempting AnalyseComplex on fold_complex_original_model_2.pdb
Time needed for AnalyseComplex: 19.89s


Obtaining file from path /home/luca3/Downloads/fold_complex_original/fold_complex_original_model_3.cif
Attempting AnalyseComplex on fold_complex_original_model_3.pdb
Time needed for AnalyseComplex: 20.13s


Obtaining file from path /home/luca3/Downloads/fold_complex_original/fold_complex_original_model_4.cif
Attempting AnalyseComplex on fold_complex_original_model_4.pdb
Time n

In [13]:
res_nob = all_results[-1]
res_nob = fix_columns(res_nob)
sel_cols = ['Pdb','Energy','Interaction','StabilityGroup1','StabilityGroup2','Interface_Residues','Clashing']
sel_res_nob = res_nob[sel_cols]

# groupby Pdb (average over all combinations of polypeptides)
sel_res_nob = sel_res_nob.groupby('Pdb').mean().reset_index()
# average over all possible models
positions = []
for i in range(np.shape(sel_res_nob)[0]):
    positions.append((sel_res_nob.iloc[i]['Pdb'][22:25]))

sel_res_nob['Pos'] = positions
# select only numeric col
num_cols = sel_res_nob.select_dtypes(include='number').columns
avg_df = sel_res_nob.groupby('Pos')[num_cols].median().reset_index()
std_df = sel_res_nob.groupby('Pos')[num_cols].std().reset_index()
avg_df


,Pos,Energy,Interaction,StabilityGroup1,StabilityGroup2,Interface_Residues,Clashing
0,196,42.307078,36.112529,115.428433,NaN,NaN,NaN
1,197,33.439315,5.875285,72.196370,NaN,NaN,NaN
2,202,33.216654,16.959785,80.331450,NaN,NaN,NaN
3,206,28.345616,6.367036,97.735625,NaN,NaN,NaN
4,234,39.111583,18.334666,84.138675,NaN,NaN,NaN
5,90_,35.858214,18.703199,91.820083,NaN,NaN,NaN
6,l_m,32.592515,10.351022,81.353292,NaN,NaN,NaN


In [14]:
std_df

,Pos,Energy,Interaction,StabilityGroup1,StabilityGroup2,Interface_Residues,Clashing
0,196,47.654140,79.255277,178.327451,NaN,NaN,NaN
1,197,41.198659,23.798264,138.247532,NaN,NaN,NaN
2,202,42.169155,28.743716,141.574066,NaN,NaN,NaN
3,206,42.684789,21.734384,157.037450,NaN,NaN,NaN
4,234,42.668342,55.553483,145.071729,NaN,NaN,NaN
5,90_,42.433861,59.690454,152.726696,NaN,NaN,NaN
6,l_m,40.294162,25.898951,142.688245,NaN,NaN,NaN


In [15]:
for i in range(np.shape(res_nob)[0]):
    print(res_nob.iloc[i]['Pdb'],res_nob.iloc[i]['Energy'])

./fold_complex_mut_pos90_vtoq_model_4.pdb 79.3032
./fold_complex_mut_pos90_vtoq_model_4.pdb 79.3032
./fold_complex_mut_pos90_vtoq_model_4.pdb 79.3032
./fold_complex_mut_pos90_vtoq_model_4.pdb 262.608
./fold_complex_mut_pos90_vtoq_model_4.pdb 262.608
./fold_complex_mut_pos90_vtoq_model_4.pdb 287.654
./fold_complex_mut_pos234_ltoc_model_2.pdb -0.069891
./fold_complex_mut_pos234_ltoc_model_2.pdb -0.554282
./fold_complex_mut_pos234_ltoc_model_2.pdb 2.27374e-13
./fold_complex_mut_pos234_ltoc_model_2.pdb -6.37058
./fold_complex_mut_pos234_ltoc_model_2.pdb -0.559126
./fold_complex_mut_pos234_ltoc_model_2.pdb -0.342422
./fold_complex_mut_pos234_ltoc_model_1.pdb nan
./fold_complex_mut_pos234_ltoc_model_1.pdb nan
./fold_complex_mut_pos234_ltoc_model_1.pdb nan
./fold_complex_mut_pos234_ltoc_model_1.pdb nan
./fold_complex_mut_pos234_ltoc_model_4.pdb nan
./fold_complex_mut_pos234_ltoc_model_4.pdb nan
./fold_complex_mut_pos234_ltoc_model_4.pdb nan
./fold_complex_mut_pos234_ltoc_model_4.pdb nan
./fol

In [16]:
res_nob = all_results[-1]
print(np.shape(res_nob))
res_nob = fix_columns(res_nob)
sel_cols = ['Pdb','Energy','Interaction','StabilityGroup1','StabilityGroup2']
sel_res_nob = res_nob[sel_cols]

positions = []
model_idx = []
for i in range(np.shape(sel_res_nob)[0]):
    positions.append((sel_res_nob.iloc[i]['Pdb'][22:25]))
    model_idx.append((sel_res_nob.iloc[i]['Pdb'][-9:-3]))

sel_res_nob['Pos'] = positions
sel_res_nob['is_repair'] = model_idx
sel_res_nob

(1120, 69)


/tmp/ipykernel_97274/3102687409.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_res_nob['Pos'] = positions
/tmp/ipykernel_97274/3102687409.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_res_nob['is_repair'] = model_idx


,Pdb,Energy,Interaction,StabilityGroup1,StabilityGroup2,Pos,is_repair
0,./fold_complex_mut_pos90_vtoq_model_4.pdb,79.30320,63.969400,262.608,NaN,90_,del_4.
1,./fold_complex_mut_pos90_vtoq_model_4.pdb,79.30320,51.281800,287.654,NaN,90_,del_4.
2,./fold_complex_mut_pos90_vtoq_model_4.pdb,79.30320,48.431300,263.516,NaN,90_,del_4.
3,./fold_complex_mut_pos90_vtoq_model_4.pdb,262.60800,945.390000,287.654,NaN,90_,del_4.
4,./fold_complex_mut_pos90_vtoq_model_4.pdb,262.60800,17.210500,263.516,NaN,90_,del_4.
...,...,...,...,...,...,...,...
1115,./fold_complex_mut_pos202_dtoc_model_0_Repair.pdb,-7.84979,120.872000,NaN,NaN,202,epair.
1116,./fold_complex_mut_pos202_dtoc_model_0_Repair.pdb,-1.11342,-0.149242,NaN,NaN,202,epair.
1117,./fold_complex_mut_pos202_dtoc_model_0_Repair.pdb,-10.93820,-41.236600,NaN,NaN,202,epair.
1118,./fold_complex_mut_pos202_dtoc_model_0_Repair.pdb,-1.28273,1.307080,NaN,NaN,202,epair.


In [17]:
sel_res_nob = sel_res_nob[sel_res_nob['is_repair'] != 'epair.']
sel_res_nob

,Pdb,Energy,Interaction,StabilityGroup1,StabilityGroup2,Pos,is_repair
0,./fold_complex_mut_pos90_vtoq_model_4.pdb,79.3032,63.96940,262.608,NaN,90_,del_4.
1,./fold_complex_mut_pos90_vtoq_model_4.pdb,79.3032,51.28180,287.654,NaN,90_,del_4.
2,./fold_complex_mut_pos90_vtoq_model_4.pdb,79.3032,48.43130,263.516,NaN,90_,del_4.
3,./fold_complex_mut_pos90_vtoq_model_4.pdb,262.6080,945.39000,287.654,NaN,90_,del_4.
4,./fold_complex_mut_pos90_vtoq_model_4.pdb,262.6080,17.21050,263.516,NaN,90_,del_4.
...,...,...,...,...,...,...,...
1109,./fold_complex_mut_pos234_ltoc_model_0.pdb,94.3452,346.70600,197.078,NaN,234,del_0.
1110,./fold_complex_mut_pos234_ltoc_model_0.pdb,94.3452,262.85800,242.723,NaN,234,del_0.
1111,./fold_complex_mut_pos234_ltoc_model_0.pdb,201.7130,120.94600,197.078,NaN,234,del_0.
1112,./fold_complex_mut_pos234_ltoc_model_0.pdb,201.7130,9.72176,242.723,NaN,234,del_0.


In [18]:
# groupby Pdb (average over all combinations of polypeptides)
sel_res_nob = sel_res_nob.drop(columns = ['Pos','is_repair'])
sel_res_nob = sel_res_nob.groupby('Pdb').mean().reset_index()
# average over all possible models
positions = []
for i in range(np.shape(sel_res_nob)[0]):
    positions.append((sel_res_nob.iloc[i]['Pdb'][22:25]))

sel_res_nob['Pos'] = positions
# select only numeric col
num_cols = sel_res_nob.select_dtypes(include='number').columns
avg_df = sel_res_nob.groupby('Pos')[num_cols].median().reset_index()
std_df = sel_res_nob.groupby('Pos')[num_cols].std().reset_index()

trgt_energy = avg_df.iloc[5]['Energy']
avg_df['DeltaEnergy'] = avg_df['Energy'] - trgt_energy
trgt_interaction = avg_df.iloc[5]['Interaction']
avg_df['DeltaInt'] = avg_df['Interaction'] - trgt_interaction
trgt_stab = avg_df.iloc[5]['StabilityGroup1']
avg_df['DeltaStab'] = avg_df['StabilityGroup1'] - trgt_stab
avg_df

,Pos,Energy,Interaction,StabilityGroup1,StabilityGroup2,DeltaEnergy,DeltaInt,DeltaStab
0,196,84.035332,90.001017,262.327167,NaN,14.752221,41.569703,32.919833
1,197,70.264739,38.704092,203.310500,NaN,0.981629,-9.727222,-26.096833
2,202,69.217991,34.157507,213.323333,NaN,-0.065120,-14.273807,-16.084000
3,206,70.690154,14.027741,213.443000,NaN,1.407043,-34.403572,-15.964333
4,234,74.757139,21.247560,220.673000,NaN,5.474029,-27.183753,-8.734333
5,90_,69.283111,48.431313,229.407333,NaN,0.000000,0.000000,0.000000
6,l_m,70.503069,30.504655,218.146167,NaN,1.219959,-17.926658,-11.261167


In [19]:
std_df['Energy'] = std_df['Energy'] / np.sqrt(5)
std_df['Interaction'] = std_df['Interaction'] / np.sqrt(5)
std_df['StabilityGroup1'] = std_df['StabilityGroup1'] / np.sqrt(5)
std_df

,Pos,Energy,Interaction,StabilityGroup1,StabilityGroup2
0,196,7.380608,40.375323,31.410649,NaN
1,197,2.615000,6.489516,2.170277,NaN
2,202,2.003229,11.853909,5.539225,NaN
3,206,1.881602,7.899648,27.345162,NaN
4,234,2.133692,29.679711,6.798615,NaN
5,90_,3.825596,31.360632,19.750046,NaN
6,l_m,1.839380,11.018069,4.206703,NaN


# Get pLLDT

In [20]:
import json
import os
import re
import pandas as pd
import numpy as np
from Bio.PDB import MMCIFParser, PDBParser

# ----------------------
# CONFIG
# ----------------------

DATA_DIR = "/home/luca3/Downloads"
ORIGINAL_DIR = "fold_complex_original"
MUTATION_DIR = "fold_complex_"
MUTATION_REGEX = r"mut_pos(\d+)"  # extract mutation position from folder name
LOCAL_WINDOW = 5  # +/- residues

'''mutation_list = ["mut_pos90_vtoq",
                 "mut_pos95_ptof",
                 "mut_pos104_dtof",
                 "mut_pos161_rtos",
                 "mut_pos196_ftoh",
                 "mut_pos197_ntoi",
                 "mut_pos200_wtoe",
                 "mut_pos202_dtoc",
                 "mut_pos203_wtop",
                 "mut_pos233_ltop",
                 "mut_pos234_ltop"
                 
                ]'''

# ----------------------
# Helpers
# ----------------------
def load_json(path):
    with open(path) as f:
        return json.load(f)
        
def extract_global_metrics(full_data_path, summary_path):
    # Get mean pLDDT from atom_plddts
    full_data = load_json(full_data_path)
    residues = full_data.get("atom_plddts", [])
    mean_plddt = sum(residues) / len(residues) if residues else None

    # PTM score from summary
    summary_data = load_json(summary_path)
    ptm_score = summary_data.get("ptm")  # or None if not there

    return mean_plddt, ptm_score

def extract_local_plddt(full_data_path, pos, window):
    data = load_json(full_data_path)
    residues = data.get("atom_plddts", [])
    start = max(0, pos - 1 - window)
    end = min(len(residues), pos + window)
    local_scores = residues[start:end]
    return sum(local_scores) / len(local_scores) if local_scores else None

def extract_ca_coords_cif(cif_file):
    """
    Extract Cα coordinates per chain from a CIF file.
    Returns a dict: {chain_id: np.array([[x,y,z], ...])}
    """
    parser = MMCIFParser()
    structure = parser.get_structure("prot", cif_file)
    coords = {}
    for model in structure:
        for chain in model:
            chain_id = chain.id
            ca_coords = []
            for res in chain:
                if "CA" in res:
                    ca_coords.append(res["CA"].get_coord())
            coords[chain_id] = np.array(ca_coords)
        break  # only first model
    return coords

def effective_strain_vector(ca_coords_ref, ca_coords_target, cutoff=13.0):
    """
    Compute Effective Strain for all residues in a structure
    relative to a reference structure.
    """
    n_residues = ca_coords_ref.shape[0]
    es_vector = np.zeros(n_residues)
    
    for i in range(n_residues):
        ref_coord = ca_coords_ref[i]
        distances = np.linalg.norm(ca_coords_ref - ref_coord, axis=1)
        neighbors = np.where(distances <= cutoff)[0]
        diff = ca_coords_target[neighbors] - ca_coords_ref[neighbors]
        es_vector[i] = np.sqrt(np.mean(np.sum(diff**2, axis=1)))
    
    return es_vector

from scipy.stats import pearsonr

def es_correlation(ca_coords_ref, ca_coords_target, cutoff=13.0):
    es_ref = effective_strain_vector(ca_coords_ref, ca_coords_ref, cutoff)
    es_target = effective_strain_vector(ca_coords_ref, ca_coords_target, cutoff)
    
    # Only compute correlation if es_ref is not constant
    if np.std(es_ref) == 0 or np.std(es_target) == 0:
        r = np.nan
    else:
        r, _ = pearsonr(es_ref, es_target)
    return r, es_ref, es_target



# ----------------------
# Compute Effective Strain
# ----------------------
def effective_strain_ca(ca_coords_wt, ca_coords_mut, mut_idx, cutoff=13.0):
    """
    Compute local Effective Strain around a mutation based on Cα distances.
    Only include neighbors within cutoff distance in WT structure.
    """
    ref_coord = ca_coords_wt[mut_idx]
    distances = np.linalg.norm(ca_coords_wt - ref_coord, axis=1)
    neighbor_idx = np.where(distances <= cutoff)[0]

    # Compute RMSD in the neighborhood
    diff = ca_coords_mut[neighbor_idx] - ca_coords_wt[neighbor_idx]
    es = np.sqrt(np.mean(np.sum(diff**2, axis=1)))
    return es

# ----------------------
# Load original reference
# ----------------------
orig_summary_file = os.path.join(DATA_DIR, ORIGINAL_DIR, f"{ORIGINAL_DIR}_summary_confidences_0.json")
orig_full_file = os.path.join(DATA_DIR, ORIGINAL_DIR, f"{ORIGINAL_DIR}_full_data_0.json")

orig_mean_plddt, orig_ptm = extract_global_metrics(orig_full_file, orig_summary_file)


# ----------------------
# Compare mutants
# ----------------------


results = []

for mut in mutation_list:
    mut_dir = MUTATION_DIR + mut
    folder_path = os.path.join(DATA_DIR, mut_dir)

    name_match = re.search(MUTATION_REGEX, mut)
    if not name_match:
        print(f"⚠️ Skipping {mut} — position not found in name")
        continue
    mut_pos = int(name_match.group(1))
    mut_idx = mut_pos - 1  # 0-based index

    # storage for per-model metrics
    per_model_metrics = {
        "mean_plddt": [],
        "tm_score": [],
        "local_plddt": [],
        "local_es": [],
        "avg_es": []
    }

    # Loop over all 5 models
    for model_idx in range(5):
        summary_file = os.path.join(folder_path, f"{mut_dir}_summary_confidences_{model_idx}.json")
        full_data_file = os.path.join(folder_path, f"{mut_dir}_full_data_{model_idx}.json")

        if not os.path.exists(summary_file) or not os.path.exists(full_data_file):
            print(f"⚠️ Missing files for {mut}, model {model_idx}")
            continue

        mean_plddt, tm_score = extract_global_metrics(full_data_file, summary_file)
        local_plddt_mut = extract_local_plddt(full_data_file, mut_pos, LOCAL_WINDOW)

        wt_cif_file = os.path.join(DATA_DIR, ORIGINAL_DIR, f"{ORIGINAL_DIR}_model_{model_idx}.cif")
        mut_cif_file = os.path.join(folder_path, f"{mut_dir}_model_{model_idx}.cif")

        wt_coords = extract_ca_coords_cif(wt_cif_file)["A"]  # assuming chain A
        mut_coords = extract_ca_coords_cif(mut_cif_file)["A"]

        r_es, es_wt_vec, es_mut_vec = es_correlation(wt_coords, mut_coords, cutoff=13.0)
        local_es_mut = effective_strain_ca(wt_coords, mut_coords, mut_idx, cutoff=13.0)

        # collect
        per_model_metrics["mean_plddt"].append(mean_plddt)
        per_model_metrics["tm_score"].append(tm_score)
        per_model_metrics["local_plddt"].append(local_plddt_mut)
        per_model_metrics["local_es"].append(local_es_mut)
        per_model_metrics["avg_es"].append(np.mean(es_mut_vec))

    # Average over models (skip None)
    avg_mean_plddt = np.nanmean([m for m in per_model_metrics["mean_plddt"] if m is not None])
    avg_tm_score = np.nanmean([m for m in per_model_metrics["tm_score"] if m is not None])
    avg_local_plddt = np.nanmean([m for m in per_model_metrics["local_plddt"] if m is not None])
    avg_local_es = np.nanmean([m for m in per_model_metrics["local_es"] if m is not None])
    avg_avg_es = np.nanmean([m for m in per_model_metrics["avg_es"] if m is not None])

    # compute std
    sd_mean_plddt = np.nanstd([m for m in per_model_metrics["mean_plddt"] if m is not None])
    sd_tm_score = np.nanstd([m for m in per_model_metrics["tm_score"] if m is not None])
    sd_local_plddt = np.nanstd([m for m in per_model_metrics["local_plddt"] if m is not None])
    sd_local_es = np.nanstd([m for m in per_model_metrics["local_es"] if m is not None])
    sd_avg_es = np.nanstd([m for m in per_model_metrics["avg_es"] if m is not None])

    # Original (WT) values — averaged across the same 5 models
    orig_per_model_plddt = []
    orig_per_model_tm = []
    orig_per_model_local_plddt = []
    for model_idx in range(5):
        orig_full_data_file = os.path.join(DATA_DIR, ORIGINAL_DIR, f"{ORIGINAL_DIR}_full_data_{model_idx}.json")
        if os.path.exists(orig_full_data_file):
            mean_plddt_orig, tm_score_orig = extract_global_metrics(
                orig_full_data_file,
                os.path.join(DATA_DIR, ORIGINAL_DIR, f"{ORIGINAL_DIR}_summary_confidences_{model_idx}.json")
            )
            orig_per_model_plddt.append(mean_plddt_orig)
            orig_per_model_tm.append(tm_score_orig)
            orig_per_model_local_plddt.append(
                extract_local_plddt(orig_full_data_file, mut_pos, LOCAL_WINDOW)
            )

    avg_mean_plddt_orig = np.nanmean(orig_per_model_plddt)
    avg_tm_score_orig = np.nanmean(orig_per_model_tm)
    avg_local_plddt_orig = np.nanmean(orig_per_model_local_plddt)

    # stds
    std_mean_plddt_orig = np.nanstd(orig_per_model_plddt)
    std_tm_score_orig = np.nanstd(orig_per_model_tm)
    std_local_plddt_orig = np.nanstd(orig_per_model_local_plddt)

    results.append({
        "mutation": mut,
        "mut_pos": mut_pos,
        "mean_plddt_mut": avg_mean_plddt,
        "std_plddt_mut": sd_mean_plddt / np.sqrt(5),
        "mean_plddt_orig": avg_mean_plddt_orig,
        "delta_mean_plddt": avg_mean_plddt - avg_mean_plddt_orig if not np.isnan(avg_mean_plddt) and not np.isnan(avg_mean_plddt_orig) else None,
        "tm_score_mut": avg_tm_score,
        "std_tm_score_mut": sd_tm_score/ np.sqrt(5),
        "tm_score_orig": avg_tm_score_orig,
        "delta_tm_score": avg_tm_score - avg_tm_score_orig if not np.isnan(avg_tm_score) and not np.isnan(avg_tm_score_orig) else None,
        "local_plddt_mut": avg_local_plddt,
        "std_local_plddt_mut": sd_local_plddt/ np.sqrt(5),
        "local_plddt_orig": avg_local_plddt_orig,
        "delta_local_plddt": avg_local_plddt - avg_local_plddt_orig if not np.isnan(avg_local_plddt) and not np.isnan(avg_local_plddt_orig) else None,
        "local_es_mut": avg_local_es,
        "std_local_es_mut": sd_local_es/ np.sqrt(5),
        "avg_es_mut": avg_avg_es,
        "std_es_mut": sd_avg_es/ np.sqrt(5),
        "local_es_orig": 0.0,
        "delta_es": avg_local_es - 0.0
    })

# Save results
df = pd.DataFrame(results)
df.to_csv("mutations_stability_comparison_avg_over_models.csv", index=False)
df


,mutation,mut_pos,mean_plddt_mut,std_plddt_mut,mean_plddt_orig,delta_mean_plddt,tm_score_mut,std_tm_score_mut,tm_score_orig,delta_tm_score,local_plddt_mut,std_local_plddt_mut,local_plddt_orig,delta_local_plddt,local_es_mut,std_local_es_mut,avg_es_mut,std_es_mut,local_es_orig,delta_es
0,mut_pos90_vtoq,90,75.336901,0.163803,75.472173,-0.135272,0.498,0.001789,0.498,0.000,78.112182,0.242842,79.866727,-1.754545,51.094109,5.244840,57.082565,3.766440,0.0,51.094109
1,mut_pos196_ftoh,196,74.998069,0.185753,75.472173,-0.474104,0.482,0.001789,0.498,-0.016,85.323636,0.246239,85.646545,-0.322909,65.827538,9.488998,64.461572,6.899402,0.0,65.827538
2,mut_pos197_ntoi,197,75.338517,0.045887,75.472173,-0.133656,0.498,0.001789,0.498,0.000,85.560545,0.154084,86.006727,-0.446182,66.915176,9.289279,64.449314,7.910210,0.0,66.915176
3,mut_pos202_dtoc,202,75.517971,0.096021,75.472173,0.045798,0.492,0.001789,0.498,-0.006,89.290364,0.261721,89.083273,0.207091,51.779499,8.738703,56.879727,6.552232,0.0,51.779499
4,mut_pos234_ltoc,234,75.604506,0.196223,75.472173,0.132333,0.500,0.000000,0.498,0.002,86.327273,0.411635,86.797091,-0.469818,49.026478,9.617260,57.466028,9.338823,0.0,49.026478
5,mut_pos206_rtoq,206,75.036774,0.105276,75.472173,-0.435399,0.486,0.002191,0.498,-0.012,86.785818,0.166037,87.398364,-0.612545,65.963913,11.106281,58.657677,7.935801,0.0,65.963913
